In [5]:
#%save Mot.py _ih[6]

In [6]:
#!/usr/bin/env python
# coding: utf-8
from nltk.stem.snowball import FrenchStemmer

class Mot:

    def __init__(self, label = "", role = "", ner = ""):
        self.label = label
        self.role = role
        self.ner = ner
        ##print(repr(label))
        stemmer = FrenchStemmer()
        self.lemma = stemmer.stem(label)
        
        
    
    def __str__(self):
        return "label = " + self.label + " role = " + self.role + " ner " + self.ner +"\n"

    def show_label(self):
        print(self.label)
    def show_role(self):
        print(self.role)
    def show_ner(self):
        print(self.ner)

if __name__ == "__main__" :
    #chaque personne est un homme ou une femme
    m1 = Mot (u"parents", "det", "0")
    #print(m1.lemma)
    m2 = Mot (u"parent", "nc", "0")
    #print(m2.lemma)
       

In [7]:
#%save parametres.py _ih[7]

In [8]:
LISTE_ETRE = [u'été', u'étée', u'étées', u'étés', u'étant', 'suis', 'es', 'est', 'sommes', u'êtes', 'sont', 'serai', 'seras', 
              'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', u'étais', u'était', 
              u'étions', u'étiez', u'étaient','fus', 'fut', u'fûmes', u'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez',
              'soient', 'fusse', 'fusses', u'fût', 'fussions', 'fussiez', 'fussent']

LISTE_AVOIR = ['a','ayant', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons',
               'aurez', 'auront', 'aurais', 'aurait', 'aurions', 'auriez', 'auraient', 'avais', 'avait', 'avions', 'aviez',
               'avaient', 'eut', u'eûmes', u'eûtes', 'eurent', 'aie', 'aies', 'ait', 'ayons', 'ayez', 'aient', 'eusse','eusses',
               u'eût', 'eussions', 'eussiez', 'eussent']


In [9]:
#%save objets_relation.py _ih[8]

In [10]:
from Mot import Mot

class Concepts:
    
    def __init__(self, mot):
            ##print("mot not null")
            self.clabel = mot.label
            self.lemma = mot.lemma
            ##print("lemme ", self.lemma)
            self.mot = mot
            self.relations_g = [] #liste des relations dans lesquelles nous avons le concept en objet_g
            self.relations_d = [] #liste des relations dans lesquelles nous avons le concept en objet_d
            self.decompositions = []
    
    
    
        
class Objets(Concepts) :
    
    def __init__(self, label, mots , is_conc, conc):
        Concepts.__init__(self, conc)
        ##print("ahhhhhhhhhhhhhhh", self.clabel)
        self.is_conc = is_conc
        self.label = label
        self.mots = mots
        return
        
    
    
        
    
    
    """
    methode qui retourne l'indice du 'cc' lorsqu'il existe
    """    
    def is_relation(self):
        indice = -1
        i = 0
        for mot in self.mots:
            if "cc" in mot.role:
                indice = i
                break;
            elif "CC" in mot.role:
                indice = i
                break;
            i += 1
        return indice
                
    """
    methode qui convertie un objet compose en Relation
    """    
        
    def get_relation(self, indice, les_objets):
        
        new1, obj1 = self.getObjet (0, indice, les_objets)
        new2, obj2 = self.getObjet (indice + 1, len (self.mots), les_objets)
        lien = self.mots[indice]
        return obj1, obj2, lien, new1, new2  
    
    def contains_adj(self,):
        for mot in self.mots:
            if(mot.role == "ADJ"):
                return True
        return False
    
    def is_adj(self,):
        if len(self.mots) == 1:
            return self.mots[0].role == "ADJ"
        return False
    def appartient(self, o_label, les_objets):
        for obj in les_objets:
            label = obj.label
            if o_label.replace(" ", "") == label.replace(" ",""):
                return True, obj
        return False, None
            
    def union_objets(self, objet2):
        newlabel= self.label + " et " + objet2.label
        newMot = self.mots
        newMot.append(Mot("et", "DET", "O"))
        newMot.extend(objet2.mots)
        is_conc,conc = isconcept(newMot)
        return Objets(newlabel, newMot, is_conc, conc)
    
    #les_objets represente la liste des objets deja traités
    def getObjet(self, indice_ini, indice_fin, les_objets): 
        #print("dkhal get objet")
        o_mot = self.mots[indice_ini : indice_fin]
        o_label = ""
        for mot in o_mot:
            if mot.label != ".":
                o_label += mot.label + " "
            else:
                o_label += mot.label
        #print("olabel", o_label)
        app, obj = self.appartient(o_label, les_objets)
        #print("aaaaaaaaaaaa",app, "obj ", str(obj))
        if app :  
            return not app, obj
        else:
            #print("getobjettttttttttttttttt ", o_mot[0])
            is_conc, conc = isconcept(o_mot)
            #print ("is_conc", conc)
             
            ##print("getobjettttttttttttttttt ",is_conc)
            return not app, Objets(o_label, o_mot, is_conc, conc)
    
    """
    cette methode permet de mettre un objet qui était à droite de la relation en objet gauche
    en mettant la première lettre en majuscule.
    """
    
    def objet_dTo_g(self):
        premier_mot = self.mots[0].label
        premier_mot = premier_mot[0].upper() + premier_mot[1:]
        self.mots[0].label = premier_mot
        self.label = self.label[0].upper() + self.label[1:]
        return self
    
    def objet_gTo_d(self):
        premier_mot = self.mots[0].label
        premier_mot = premier_mot[0].lower() + premier_mot[1:]
        self.mots[0].label = premier_mot
        self.label = self.label[0].lower() + self.label[1:]
        return self
        
    
    def __str__(self):
        
        res = "label " + self.label +" \nmots["
        for mot in self.mots:
            res += str(mot) + " , "
        res += "]"
        return res;
    
    def affichage(self): 
        
        self.show_label()
        self.show_mots()

        
    def show_label(self):
        print(self.label)
        
    def show_mots(self):
        for mot in self.mots:
            print(str(mot)) 
   

"""
Cette classe represente une relation est sous la forme objet_g ----> lien -----> objet_d
elle est definie par deux objets de la classe Objets objet_g et objet_d
et d'un lien qui est un Mot
"""
class Relation:
    
    #constructeur
    def __init__(self, objet_g, objet_d, lien, ponctuation, label, sous_relation):
        self.objet_g = objet_g
        self.objet_d = objet_d
        self.lien = lien
        self.label = label
        self.sous_relation = sous_relation
        self.ponctuation = ponctuation
        #self.les_objets = []
        self.origine = []
        
    def __str__(self):
        res = self.label + "\n" + str(self.objet_g) + "\n--->\n" + str(self.lien) + "\n--->\n" + str(self.objet_d)
        res += "\nsous-relation\n\t" + str(self.sous_relation) + "\norigine\n\t"
        for o in self.origine:
            res += str(o.relationToligne()) + "\t"
        return res
    
    
    
    
    """
    
    methode qui retourne True si la relation est simple c-a-d contient un seul un mot verbe un mot 
    sinon elle retourne False
    
    """
    def relationToligne(self):
        return self.objet_g.label + self.lien.label +" "+ self.objet_d.label + self.ponctuation.label
    
    def isSimple(self):
        #print("hhhhhhh")
        return (self.objet_g.is_relation() == -1) and (self.objet_d.is_relation() == -1)
            
    """
    cette fonction sert à convertir une phrase quelconque en une relation
    """
def getRelation_verbe(label, mots, roles, ners, les_objets):
    #lesmots represente la liste des mots sous format d'objets de la classe Mot
    lesmots = []
    for i in range(len(mots)):
        lesmots.append(Mot(mots[i], roles[i], ners[i]))
    is_conc, conc = isconcept(lesmots)
    #print("premier isconcept")
    objet = Objets(label, lesmots, is_conc, conc)
    try:
        indice = roles.index("V")
        new, objet_g = objet.getObjet(0, indice, les_objets)
        #print("OBJET G")
        try:
            indice_ponc = roles.index("PUNC")
        except:
            indice_ponc = len(roles)-1
        if new:
            les_objets.append(objet_g)
        new, objet_d = objet.getObjet(indice+1, indice_ponc,  les_objets)
        if new:
            les_objets.append(objet_g)
        lien = lesmots[indice]
        ponctuation = lesmots [indice_ponc]
        ind = objet_d.is_relation()
        if ind != -1 :
            objg, objd, lien1, new1, new2 = objet_d.get_relation(ind, les_objets)
            if new1:
                les_objets.append(objg)
            if new2:
                les_objets.append(objd)
            sous_relation = Relation(objg, objd, lien1, None , "conj de coordination", None)
        else:
            sous_relation = None
        return les_objets, Relation(objet_g, objet_d, lien, ponctuation, "verbe", sous_relation)
    except Exception as e:
        message = str(e)
        ##print(message)
        raise ValueError("il n'y a pas de verbe!!! " + message) 
def isconcept(mots):
    i = 0
    le_conc = None
    if len(mots) == 1:
        #print("len 1")
        if mots[0].role == "NPP":
            if mots[0].ner == "PERSON":
                #print("dans person")
                le_conc = Mot("personne", "NC", "O")
                #print ("leconc ", le_conc.clabel)
            elif mots[0].ner == "LOCATION":
                le_conc = Mot("lieu", "NC", "O")
            else:
                le_conc = Mot("inconnu", "null", "O")
            return False, le_conc
        elif mots[0].role == "DET":
            return False, None
        else:
            return True, mots[0]
    else:
        for mot in mots: # traitement du cas où nous avons un  nom commun dans la liste des mots
            if mot.role == "NC" or mot.role == "N":
                le_conc = mot
                label = ""
                for lab in range(0, i):
                    label += mots[lab].label + " "
                if label in ['tout ', 'chaque ', 'toute ', 'tous les ', "toutes les ", "tous ", "toutes "]:
                    return True, le_conc
                return False, le_conc
            i +=1
        for mot in mots: #traitement du cas où nous n'avons pas de nom commun mais un adjectif
            if mot.role == "ADJ":
                le_conc = mot
                return False , le_conc  
    return False, le_conc

if __name__ == "__main__" :
        #chaque personne est un homme ou une femme

    ners = ['PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
    roles = ['NPP', 'V', 'DET', 'NC', 'CC', 'DET', 'NC', 'PUNC']
    words = ['Patrick', 'est', 'un', 'homme', 'ou', 'une', 'femme', '.']
    label = "Patrick est un homme ou une femme."
    les_objets = []
    liste_objet, r = getRelation_verbe(label, words, roles, ners, les_objets)
    print(str(r.objet_g.is_conc), "son concept", r.objet_g.clabel)
    #print(str(r.relationToligne()))
    

    """
    
    cette classe 
    
    """



('False', 'son concept', 'personne')


In [16]:
#%save gestion.py _ih[14]

In [14]:
from Mot import Mot
from objets_relation import Relation, Objets, Concepts, isconcept
from parametres import *


class Gestionnaire_Relation:
    
    #constructeur
    def __init__(self, ):
        self.liste_relations = []
        self.liste_objets = []
        self.conclusions = []
        self.hypotheses = []
        self.liste_concepts = []
        self.decomposition = {}
        self.objets_d_relations = {}
        self.objets_g_relations = {}
        
    """ 
    cette fonction permet de mettre toutes les tettres d'une phrase en miniscule sauf si c'est un nom propore
    """
    def pre_traitement(self,label, mots, roles):

        for i in range(len(mots)):
            if roles[i] != "NPP":
                m = mots[i].lower()
                mots[i] = m
        label=""
        for mot in mots:
            label+= mot + " "
        label = label[0: len(label)-1]
        return label
        
    def ajouter_relation(self, label, words, roles, ners) :
        
        relation = self.getRelation_verbe(label, words, roles, ners)
        self.liste_relations.append(relation)
        return relation

    
    def concept_existant(self, conc_lemma):
        for c in self.liste_concepts:
            #print(c)
            if c.lemma == conc_lemma:
                return True, c
        return False, None
    
    def ajout_objet(self, indice_ini, indice_fin, objets):
        new, objet = objets.getObjet(indice_ini, indice_fin, self.liste_objets)
        print("new ", new)
        exist, conc = self.concept_existant(objet.lemma)
        #print("exist", exist)
        if exist == False:
            print ("the_coonc", objet.lemma)
            self.liste_concepts.append(Concepts(objet.mot))
        """
        else:
            #self, label, mots , is_conc, conc
            new_obj = Objets(objet.label, objet.mots, objet.is_conc, objet.mot)
            objet = new_obj
        """
        if new:
            self.liste_objets.append(objet)
            
        return objet
        
    def ajout_obj_sousrelation(self, objet, new):
        exist, conc = self.concept_existant(objet.lemma)
        if exist == False:
            self.liste_concepts.append(Concepts(objet.mot))
        """
        else:
            #self, label, mots , is_conc, conc
            new_obj = Objets(objet.label, objet.mots, objet.is_conc, objet.mot)
            objet = new_obj     
        """
        if new:
            self.liste_objets.append(objet)
            
        return objet
        
        
    """
    cette fonction sert à convertir une phrase quelconque en une relation
    """
    def getRelation_verbe(self, label, mots, roles, ners):
        
        #lesmots represente la liste des mots sous format d'objets de la classe Mot
        lesmots = []
        for i in range(len(mots)):
            lesmots.append(Mot(mots[i], roles[i], ners[i]))
        #print("iiiiiiiiiiiiiiii",len(lesmots))
        is_conc, conc = isconcept(lesmots)   
        objet = Objets(label, lesmots, False, conc)
        try:
            indice = roles.index("V")
            #print("indice", indice)
            objet_g = self.ajout_objet(0, indice, objet)
            #print("ajout g ok")
            try:
                indice_ponc = roles.index("PUNC")
            except:
                indice_ponc = len(roles)-1
            #print("indice_ponc", indice_ponc)
            objet_d = self.ajout_objet(indice+1, indice_ponc, objet)
            
            #print("ajout d ok")
            lien = lesmots[indice]
            ##print("oooooooooooooooooooooooooooo", indice_ponc, "len len", len(lesmots))
            ponctuation = lesmots [indice_ponc]
            ind = objet_d.is_relation()
            if ind != -1 :
                objg, objd, lien1, new1, new2 = objet_d.get_relation(ind, self.liste_objets)
                obj_g = self.ajout_obj_sousrelation(objg, new1)
                obj_d = self.ajout_obj_sousrelation(objd, new2)
                sous_relation = Relation(obj_g, obj_d, lien1, None , "conj de coordination", None) 
            else:
                sous_relation = None
            r = Relation(objet_g, objet_d, lien, ponctuation, "verbe", sous_relation)
            if ind != -1 :    
                if(lien1.lemma == "ou"):
                    self.decomposition[objet_g] = [sous_relation, r]
            if objet_g.is_conc :
                self.traitement_relation_concept(objet_g, r,"relation_g")
            if objet_d.is_conc :
                self.traitement_relation_concept(objet_d, r, "relation_d")
            
            return r
        except Exception as e:
            raise ValueError("il n'y a pas de verbe!!! ", str(e))
        return
    
    def traitement_relation_concept(self, objet, relation, position):
        if objet.is_conc:
            if position == "relation_g":
                objet.relations_g.append(relation)
            elif position == "relation_d":
                objet.relations_d.append(relation)            
            
    def traitement_avoir(self, relation):
        ##print ("rrrr", relation)
        if(relation == None):
            lien = Mot("est partie de", "V", "O")
            return lien
        elif relation.lien.label == "et":
            lien = Mot("sont partie de", "V", "O")
            return lien
        else:
            return None
    
    
    
    def traitement_NPropore(self, relation):
        
        return None
    
        
    def concatene_objets (self, objs):
        newMot = objs[0].mots
        vergule = Mot(",", "PUNC", "O")
        newlabel = objs[0].label
        for i in range(1, len(objs)-1):
            newMot.append(vergule)
            newMot.extend(objs[i].mots)
            newlabel += ", " + objs[i].label
        newMot.append(Mot("ou", "CC", "O"))
        newMot.extend(objs[len(objs)-1].mots)
        newlabel += "ou " + objs[len(objs)-1].label
        is_conc, conc = isconcept(newMot)
        
        return Objets(newlabel, newMot, is_conc, conc)
    
    """
    cette fonction sert à traiter la règle:
    Si obj1 est obj2 ou obj3 ET obj2 est obj21 ou obj22 ET obj3 est obj31 ou obj32 Alors
    obj1 est obj21 , obj22, obj31 ou obj32
    """
    def decomposer(self, new_relation):
        if new_relation.sous_relation != None:
            if(new_relation.objet_g in self.decomposition.keys()):
                #print("il est dans les keys")
                relation_pere, pere, objet_frere = self.pere_de(new_relation.objet_g)
                #print("pere "+str(pere)+"\n"+str(relation_pere))
                #print("frere "+str(objet_frere)+"\n")
                if  pere != None:
                    if(objet_frere in self.decomposition.keys()):
                        sous1 = new_relation.sous_relation
                        sous2, relation_frere = self.decomposition[objet_frere]

                        print (sous2.lien)
                        print(sous1.lien)
                        newobjet = self.concatene_objets([sous1.objet_g, sous1.objet_d, sous2.objet_g, sous2.objet_d])
                        self.liste_objets.append(newobjet)
                        ind = newobjet.is_relation()
                        objg, objd, lien1, new1, new2 = newobjet.get_relation(ind, self.liste_objets)
                        if new1:
                            self.liste_objets.append(objg)
                        if new2:
                            self.liste_objets.append(objd)
                        sous_relation = Relation(objg, objd, lien1, None , "conj de coordination", None)
                        r = Relation(pere, newobjet, new_relation.lien, new_relation.ponctuation, "verbe", 
                                    sous_relation)
                        #print("ooooooooooooooooooo "+str(relation_pere)+"\n aaaaaaaaaaaaa "+ str(relation_frere) + 
                        #"\n bbbbbbbbbbbbb "+str(new_relation))
                        r.origine.extend([relation_pere, relation_frere, new_relation])
                        return r 
        return None
    
    
    """
    cette fonction permet de detecter si un objet fait partie de la décomposition d'un autre objet et
    de retourner cet objet père  
    """
    def pere_de(self, objet):
        ##print("dkhal pere_de")
        for key, relation in self.decomposition.items():
            ##print("key "+ str(key) + "relation 0" +str(relation[0]) + "relation 1" +str(relation[1]))
            if(objet == relation[0].objet_g):
                return relation[1], key,  relation[0].objet_d
            if (objet == relation[0].objet_d):
                return relation[1], key,  relation[0].objet_g
        return None, None, None
    
    
    
    """
    cette fonction sert à traiter la règle:
    Si obj1 est obj2 ET obj2 est obj3 Alors
    obj1 est obj3
    """
    def chercher_ascendant(self, new_relation):
        #print("dkhal lahna"+ new_relation.relationToligne())
        for relation in self.liste_relations:
            if relation.lien.label in LISTE_ETRE:
                #print("rd", relation.objet_d, "nrd", new_relation.objet_d, "rg", relation.objet_g,"nrg", new_relation.objet_g)
                if relation.objet_d == new_relation.objet_g:
                    r = Relation(relation.objet_g, new_relation.objet_d, relation.lien, relation.ponctuation, "verbe", 
                                new_relation.sous_relation)
                    r.origine.extend([relation,new_relation])
                    return r
                if relation.objet_g == new_relation.objet_d:
                    #print("dkhal lahna")
                    r = Relation(new_relation.objet_g, relation.objet_d, relation.lien, relation.ponctuation, "verbe", 
                                None)
                    r.origine.extend([relation,new_relation])
                    return r
        return None
    
    """
    cette fonction  sert à traiter la règle:
    Si obj1 est obj2 ou obj3   Et   obj2 est obj4    et  obj3 est obj5 Alors
    
    obj1 est obj4 ou obj5
    """
    def nouvelle_decomposition (self, new_relation):
        relation_pere, pere, objet_frere = self.pere_de(new_relation.objet_g)
        if pere!= None:
            for relation in self.liste_relations:
                if (relation.lien.label in LISTE_ETRE) and (relation.objet_g == objet_frere) and (relation.sous_relation == None):
                    new_obj = self.concatene_objets([relation.objet_d, new_relation.objet_d])
                    self.liste_objets.append(new_obj)
                    lien1 = Mot("ou", "CC", "O")
                    sous_relation = Relation(relation.objet_d, new_relation.objet_d, lien1, None , "conj de coordination", None)
                    r = Relation(pere, new_obj, relation.lien, relation.ponctuation, "verbe", sous_relation)
                    r.origine.extend([relation_pere, relation, new_relation])
                    return r
        return None
                    
            
            
        
        
        
        
        
    """
    cette fonction sert à traiter la règle:
    Si obj1 est obj2 Et obj3 est obj2 Alors
    obj1 et Obj3 sont obj2
    """
    def union(self, new_relation):
        for relation in self.liste_relations:
            if relation.lien.label in ['est']:
                if(relation != new_relation):
                    if relation.objet_d == new_relation.objet_d:
                        newobj = relation.objet_g.union_objets(new_relation.objet_g)
                        newlien = Mot("sont", "V", "O")
                        r = Relation(newobj, new_relation.objet_d, newlien, relation.ponctuation, "verbe", 
                                    new_relation.sous_relation)
                        r.origine.extend([relation,new_relation])
                        return r
                else:
                    return None
        return None
    
    """
    cette fonction sert à traiter la règle:
    Si obj1 est obj2 ou obj3  ET  obj3 est obj2 Alors
    obj3 est probablement Obj1 
    """
    def simple_hypothese(self, relation):
        relation_pere, pere, objet_frere = self.pere_de(relation.objet_d)
        ##print("ooooooooooooooooooooooo",str(pere))
        if pere != None:
            lien = Mot("est probablement", "V", "O")
            r = Relation(relation.objet_g, pere, lien, relation.ponctuation, "verbe", None)
            r.origine.extend([relation_pere, relation])
            return r
        else:
            return None
    
    
    """
    cette fonction sert à traiter la règle:
    Si obj1 est obj2 ou obj3  ET  obj4 est obj5 ou obj2 Alors
    obj4 est probablement Obj1 
    """
    def complex_hypothese(self, relation):
        sous_relation, r = self.decomposition[relation.objet_g]
        relation_pere_d, pere_d, objet_frere_d = self.pere_de(sous_relation.objet_d)
        if (pere_d != None) and not(pere_d == relation.objet_g):
            lien = Mot("est probablement", "V", "O")
            res = Relation(relation.objet_g, pere_d, lien, relation.ponctuation, "verbe", None)
            res.origine.extend([relation_pere_d, relation])
            return res
        relation_pere_g, pere_g, objet_frere_g = self.pere_de(sous_relation.objet_g)
        if (pere_g != None) and not(pere_g == relation.objet_g):
            lien = Mot("est probablement", "V", "O")
            res = Relation(relation.objet_g, pere_g, lien, relation.ponctuation, "verbe", None)
            res.origine.extend([relation_pere_g, relation])
            return res
        return None
        
        
        
    def getHypothese(self, relation):
        nouvelles_hypotheses = []
        new = False
        if(not relation.objet_d.is_adj()):
            if relation.lien.label in LISTE_ETRE:   #il faut que ça soit la liste du verbe etre
                if(relation.sous_relation == None):
                    r = self.simple_hypothese(relation)
                    ##print(r.relationToligne())
                    if r!=None:
                        self.hypotheses.append(r)
                        nouvelles_hypotheses.append(r)
                        new = True
                else:
                    r = self.complex_hypothese(relation)
                    if r!=None :
                        ##print(str(r.relationToligne()))
                        self.hypotheses.append(r)
                        nouvelles_hypotheses.append(r)
                        new = True
        while new:
            r_new = self.sous_hypotheses(r)
            if r_new == None:
                new = False
            else:
                r = r_new
                self.hypotheses.append(r)
                nouvelles_hypotheses.append(r)   
        return nouvelles_hypotheses
    
    
    
    def contient (self, obj_g, obj_d, lien):
        for relation in self.hypotheses:
            if(relation.objet_g == obj_g) and (relation.objet_d == obj_d) and (relation.lien.label == lien.label):
                return True
        return False
    """
    cette fonction genere les hypotheses respectant la regle
    Si obj1 est obj2 Ou obj3 ET obj2 est problablement obj4 Alors
    obj1 est probablement obj4.
    """
    
    def sous_hypotheses(self, new_relation):
        relation_pere, pere, objet_frere = self.pere_de(new_relation.objet_g)
        if pere != None:
            lien = Mot("est probablement", "V", "O")
            if not self.contient(pere, new_relation.objet_d, lien):
                r = Relation(pere, new_relation.objet_d, lien, new_relation.ponctuation, "verbe", None)
                r.origine.extend([relation_pere, new_relation])
                ##print(str(r.relationToligne()))
                self.hypotheses.append(r)
                return r
            else:
                return None
        else:
            return None
        #return self.hypotheses
                    
    
    def getConclusions(self, relation):
        nouvelles_conclusion = []
        new = False
        if(not relation.objet_d.is_adj()):
            if relation.lien.label in LISTE_AVOIR:   
                ##print ("traitement avoir")
                conc = self.traitement_avoir(relation.sous_relation)
                if conc != None:
                    ##print("coooooooonnnnnnn", str(conc))
                    ##print(str(relation.objet_d.objet_dTo_g()))
                    ponct = Mot(".", "PUNC", "O")
                    r = Relation(relation.objet_d.objet_dTo_g(), relation.objet_g.objet_gTo_d(), conc, ponct, conc.label, None)
                    nouvelles_conclusion.append(r)
                    self.conclusions.append(r)
                    self.liste_relations.append(r)
                    new = True

            elif relation.lien.label in LISTE_ETRE:   
                # recherche ascendants
                r = self.chercher_ascendant (relation)
                print("rrrrrrrrrrr",r)
                if r != None:
                    ##print (r.relationToligne())
                    self.conclusions.append(r)
                    self.liste_relations.append(r)
                    nouvelles_conclusion.append(r)
                    new = True
                #else:
                    ##print ("pas d'ascendants")
                r = self.union(relation)
                if r!=None:
                    ##print (r.relationToligne())
                    ##print (str(r))
                    self.conclusions.append(r)
                    self.liste_relations.append(r)
                    nouvelles_conclusion.append(r)
                    new = True
                #else:
                    ##print ("pas d'unions")
                r = self.nouvelle_decomposition(relation)
                if r!=None:
                    ##print (r.relationToligne())
                    ##print (str(r))
                    self.conclusions.append(r)
                    self.liste_relations.append(r)
                    nouvelles_conclusion.append(r)
                    new = True
                #else:
                    ##print ("pas de nouvelle decomposition")
                r = self.decomposer(relation)
                if r!=None:
                    ##print (r.relationToligne())
                    ##print (str(r))
                    self.conclusions.append(r)
                    self.liste_relations.append(r)
                    nouvelles_conclusion.append(r)
                    new = True
                #else:
                    ##print ("pas de doomposition multiple")
            else:
                print ("je ne connais pas ce verbe !!!")
        else:
            print("contient un adjectif")
        return nouvelles_conclusion
                    
             
if __name__ == "__main__" :
    
    #chaque personne est un homme ou une femme
    

    ners = ['O', 'O', 'O', 'O', 'O', 'O','O', 'O', 'O']
    roles = ['DET', 'NC', 'V', 'NC', 'DET','CC', 'DET','NC', 'PUNC']
    words = ['Chaque', 'societe', 'a', 'des', 'hommes', 'et', 'des','femmes', '.']
    label = "Chaque societe a des hommes et des femmes."
    gestion = Gestionnaire_Relation()
    label = gestion.pre_traitement(label, words, roles)
    #print("ph1")
    r1 = gestion.ajouter_relation(label, words, roles, ners)   
    print(r1.objet_g.relations_g[0].relationToligne())
    
    ners = ['O', 'O', 'O', 'O', 'O', 'O','O', 'O', 'O']
    roles = ['DET', 'NC', 'V', 'NC', 'DET','CC', 'DET','NC', 'PUNC']
    words = ['Chaque', 'societe', 'a', 'des', 'enfants', 'et', 'des','parents', '.']
    label = "Chaque societe a des enfants et des parents."
    gestion = Gestionnaire_Relation()
    label = gestion.pre_traitement(label, words, roles)
    #print("ph1")
    r2 = gestion.ajouter_relation(label, words, roles, ners)   
    print(r1.objet_g == r2.objet_g)
    print(len(r2.objet_g.relations_g), "\n elem 0",r2.objet_g.relations_g[0].relationToligne())
    #print(len(gestion.decomposition))
    #print("conc1")
    """
    gestion.getConclusions(r)


    ners = ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
    roles = ['DET', 'NC', 'V', 'DET', 'NC', 'CC', 'DET', 'NC', 'PUNC']
    words = ['Chaque', 'personne', 'est', 'un', 'homme', 'ou', 'une', 'femme', '.']
    label = "Chaque personne est un homme ou une femme."
    gestion = Gestionnaire_Relation()
    label = gestion.pre_traitement(label, words)
    r = gestion.ajouter_relation(label, words, roles, ners)   
    #print("okokokokokokok", len(gestion.decomposition))
   
    #gestion.getConclusions()
    #print("hyp1")
    gestion.getHypothese(r)
    
    ners = ['O', 'O', 'O', 'O', 'O', 'O']
    roles = ['DET', 'NC', 'V', 'DET', 'NC', 'PUNC']
    words = [u'Chaque', u'père', u'est', u'un', u'homme', '.']
    label = u"Chaque père est un homme."
    label = gestion.pre_traitement(label, words)
    r = gestion.ajouter_relation(label, words, roles, ners)   
    #print(len(gestion.decomposition))
    ##print("conc1")
    #gestion.getConclusions()
    #print("hyp2")
    #print("okokokokokokok", len(gestion.decomposition))
    gestion.getHypothese(r)
    
 
    ners = ['O', 'O', 'O', 'O', 'O', 'O']
    roles = ['DET', 'NC', 'V', 'DET', 'NC', 'PUNC']
    words = ['Un', 'homme', 'est', 'une', 'personne', '.']
    label = "Un homme est une personne."
    label = gestion.pre_traitement(label, words)
    gestion.ajouter_relation(label, words, roles, ners)
    #print("conc3")
    gestion.getConclusions()
    ners = ['O', 'O', 'O', 'O', 'O', 'O']
    roles = ['DET', 'NC', 'V', 'DET', 'NC', 'PUNC']
    words = ['Chaque', 'frère', 'est', 'un', 'homme', '.']
    label = "Chaque père est un homme."
    label = gestion.pre_traitement(label, words)
    gestion.ajouter_relation(label, words, roles, ners)
    #print("conc4")
    gestion.getConclusions()
    ners = ['O', 'O', 'O', 'O', 'O']
    roles = ['DET', 'NC', 'V', 'ADJ', 'PUNC']
    words = ['Chaque', 'femme', 'est', 'féminine', '.']
    label = "Chaque femme est féminine."
    label = gestion.pre_traitement(label, words)
    gestion.ajouter_relation(label, words, roles, ners)
    #print("conc5")
    gestion.getConclusions()
    #print(len(gestion.decomposition))
    ners = ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
    roles = ['DET', 'NC', 'V', 'DET', 'NC', 'CC', 'DET', 'NC', 'PUNC']
    words = ['Chaque', 'personne', 'est', 'un', 'homme', 'ou', 'une', 'femme', '.']
    label = "Chaque personne est un homme ou une femme."
    label = gestion.pre_traitement(label, words)
    gestion.ajouter_relation(label, words, roles, ners)
    #print("conc6")
    gestion.getConclusions()
    ners = ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
    roles = ['DET', 'NC', 'V', 'DET', 'NC', 'CC', 'DET', 'NC', 'PUNC']
    words = ['Chaque', 'enfant', 'est', 'un', 'fils', 'ou', 'une', 'fille', '.']
    label = "Chaque enfant est un fils ou une fille."
    label = gestion.pre_traitement(label, words)
    gestion.ajouter_relation(label, words, roles, ners)
    #print("conc7")
    gestion.getConclusions()
    #print(len(gestion.decomposition))
    ners = ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
    roles = ['DET', 'NC', 'V', 'DET', 'N', 'CC', 'DET', 'NC', 'PUNC']
    words = ['Un', 'fils', 'est', 'un', 'garçon', 'ou', 'un', 'homme', '.']
    label = "Un fils est un garçon ou un homme."
    label = gestion.pre_traitement(label, words)
    gestion.ajouter_relation(label, words, roles, ners)
    #print("conc8")
    gestion.getConclusions()
    #print(len(gestion.decomposition))
    ners = ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
    roles = ['DET', 'NC', 'V', 'DET', 'NC', 'CC', 'DET', 'NC', 'PUNC']
    words = ['Une', 'fille', 'est', 'une', 'jeune-fille', 'ou', 'une', 'femme', '.']
    label = "Une fille est une jeune-fille ou une femme."
    label = gestion.pre_traitement(label, words)
    gestion.ajouter_relation(label, words, roles, ners)
    #print("conc8")
    gestion.getConclusions()
    #print(len(gestion.decomposition))
 
    """
                    
                    
                
                
    
    
    

('new ', True)
('the_coonc', u'societ')
('new ', True)
('the_coonc', u'de')
chaque societe a des hommes et des femmes .
('new ', True)
('the_coonc', u'societ')
('new ', True)
('the_coonc', u'de')
False
(1, '\n elem 0', 'chaque societe a des enfants et des parents .')


In [13]:
import time, codecs
from MonCoreNLP import lecture_fichier 
from MonCoreNLP import get_enhancedPlusPlusDependencies 
#from gestion import Gestionnaire_Relation


def generer_tous_arbres (path):
    enhanced = {}
    enhanceds = []
    arbre = {}
    arbres = []
    ner = []
    roles = []
    words = []
    relations = []

    datum = ""
    donnees = lecture_fichier(path)
    gestion = Gestionnaire_Relation()

    for datum in donnees:
        premierMoty = "ROOT"
        print("###############" + datum + "###############")
        enhanced, ner, roles, words = get_enhancedPlusPlusDependencies(datum)
        #print ("&&&&&&&&&&&&&&&&&&&&&&&&& enhenced \n" + str(enhanced))
        #print ("&&&&&&&&&&&&&&&&&&&&&&&&& ner \n" + str(ner))
        print ("&&&&&&&&&&&&&&&&&&&&&&&&& roles \n" + str(roles))
        #print ("&&&&&&&&&&&&&&&&&&&&&&&&& words \n" + str(words))
        label = gestion.pre_traitement(datum, words, roles)
        r = gestion.ajouter_relation(label, words, roles, ner)
        print("conlusions \n")
        conc = gestion.getConclusions(r)
        for c in conc:
            print(c.relationToligne())
        print("hypotheses \n")
        hyp = gestion.getHypothese(r)
        for h in hyp:
            print(h.relationToligne())
       

        #relation = Relation.getRelation_verbe(datum, words, roles, ner)
        ##print(str(relation))
        #relations.append(relation)
        #afaire!!!!!
        #relations.hypotheses retourne les hypothèses
        #relations.conclusions retourne les conclusions
        """

        ********   pour le moment pas besoin des arbres   ***************


        arbre = creationArbreDependanceNom (enhanced)
        m={}
        affichageArbre (premierMoty, arbre, m)
        enhanceds.append(enhanced)
        arbres.append(arbre)
        enhanced = {}
        arbre = {}
        """
        datum = ""

        #print("###############################################################")
    """vider l'esprit
    """

    #gestion = Gestionnaire_Relation()
    """
     self.liste_relations = []
        self.liste_objets = []
        self.conclusions = []
        self.hypotheses = []
        self.liste_concepts = []
        self.decomposition = {}
    """
    #print(len(gestion.liste_objets ))
#generer_tous_arbres("./definition de famille.txt")
generer_tous_arbres("./definition de famille (simple).txt")
#generer_tous_arbres("./Connaissance des organismes.txt")

###############Une personne est un homme ou une femme.###############
&&&&&&&&&&&&&&&&&&&&&&&&& roles 
[u'DET', u'NC', u'V', u'DET', u'NC', u'CC', u'DET', u'NC', u'PUNC']
('new ', True)
('the_coonc', u'person')
('new ', True)
('the_coonc', u'homm')
conlusions 

('rrrrrrrrrrr', None)
hypotheses 

###############Une femme est féminine.###############
&&&&&&&&&&&&&&&&&&&&&&&&& roles 
[u'DET', u'NC', u'V', u'ADJ', u'PUNC']
('new ', False)
('new ', True)
('the_coonc', u'f\xe9minin')
conlusions 

contient un adjectif
hypotheses 

###############Un homme est masculin.###############
&&&&&&&&&&&&&&&&&&&&&&&&& roles 
[u'DET', u'NC', u'V', u'ADJ', u'PUNC']
('new ', False)
('new ', True)
('the_coonc', u'masculin')
conlusions 

contient un adjectif
hypotheses 

###############Une famille a parents et enfants.###############
&&&&&&&&&&&&&&&&&&&&&&&&& roles 
[u'DET', u'NC', u'V', u'NC', u'CC', u'NC', u'PUNC']
('new ', True)
('the_coonc', u'famill')
('new ', True)
('the_coonc', u'parent')
conlusions 